# Automated Data Collection (no manual map change) -- BROKEN

The original method used in this package required a manual map change between base and annotated collection. This notebook forgoes this manual change. For explanation on the methods used, please see `bngseg.ipynb`.

In [39]:
# setup
from beamngpy import BeamNgpy, Scenario, Vehicle
from beamngpy.sensors import Camera
from pathlib import Path
from tqdm.auto import tqdm

from bngseg.annotation import AnnotationScenario, AnnotationCar, CameraPosition
import bngseg

home_dir = r'C:\Users\jazzy\Desktop\BeamNG' # change this to correct directory
user_dir = r'C:\Users\jazzy\Desktop\BeamNG' # change this to correct directory

bng = util.quick_start(home_dir, user_dir)

In [32]:
import random
def generate_random_point(points, radius=4):  # not optimized
    """Generate random point within a given radius from any point within points list."""
    point = random.choice(points)
    offsets = (random.uniform(-radius, radius), random.uniform(-radius, radius), 0.5)
    rot_quat = (0, 0, 0, 0)
    return (
        point[0] + offsets[0],
        point[1] + offsets[1],
        point[2] + offsets[2],
    ), rot_quat

In [34]:
# variables
points_bank = {
    "rb_ks_monza": [(-177.105, -107.766, 154.945)]
}

n = 10 # per car per scenario
scenarios = [
    AnnotationScenario(
        "rb_ks_monza", 
        "rb_ks_monza_annotated", [
            AnnotationCar(
                '2022__4_navaro_ssr_700_indycar', 
                [
                    CameraPosition((-0.3, 1, 2), (0, -1, 0), (0, 0, 1)),
                ]
            ),
        ],
        [generate_random_point(points_bank["rb_ks_monza"]) for _ in range(n)],
    ),
]

TypeError: __init__() missing 1 required positional argument: 'cars'

In [40]:
import importlib
importlib.reload(annotationscustom)

SyntaxError: invalid syntax (annotation.py, line 76)

## Ensure annotation works

In [ ]:
test = scenarios[0]
test, car = bng.quick_scenario(bng, test.base_map, scenarios.cars[0].car_model, test.locations[0][0], test.locations[0][1])
camera = test_car.attach_cameras([((-0.3, 1, 2), (0, -1, 0))], car)[0]
bngseg.poll_and_show_images(camera)

## Collect data (script)

In [ ]:
# random cameras currently not supported, just takes first spec

def gather_data(scenario, car, states, annotated=False):
    last_car = None
    i = 0
    images = {}
    for state in tqdm(states):
        if state[0] != last_car:
            scenario.remove_vehicle(car)
            car = Vehicle(state[0], model=state[0])
            scenario.add_vehicle(car, pos=state[1][0], rot_quat=state[1][1])
            cameras = car.attach_cameras(state[2]) # random cameras not currently implemented
            last_car = state[0]
        else:
            car.teleport(state[1][0], rot_quat=state[1][1])

        for j, cam in enumerate(cameras):
            base_image = cam.get_full_poll_request()[
                "colour" if not annotated else "annotation"
            ]
            name = "{}_{}".format(i, j)
            images[name] = base_image
        i += 1
    return images
    

s += 1
for spec in (pbar := tqdm(scenarios, total=len(scenarios)):
    pbar.set_description(f"Processing {scenario.base_map}")
    
    scenario, car = bng.quick_scenario(bng, spec.base_map, spec.cars[0].car_model, spec.locations[0][0], spec.locations[0][1])
    base_images = gather_data(scenario, car, spec)
    
    scenario, car = bng.quick_scenario(bng, spec.annotated_map, spec.cars[0].car_model, spec.locations[0][0], spec.locations[0][1])
    annotated_images = gather_data(scenario, car, spec.history.locations, annotated=True)

    folder = bngseg.save_images(
        bngseg.pair_images(base_images, annotated_images)
    )

    spec.history.save(folder)
    

        

In [35]:
import bngseg.annotation as annotation

n = 10
scenarios = [
    annotation.AnnotationScenario(
        "rb_ks_monza", 
        "rb_ks_monza_annotated", [
            annotation.AnnotationCar(
                '2022__4_navaro_ssr_700_indycar', 
                [
                    annotation.CameraPosition((-0.3, 1, 2), (0, -1, 0), (0, 0, 1)),
                ]
            ),
            annotation.AnnotationCar(
                'boliva', 
                [
                    annotation.CameraPosition((-0.3, 1, 2), (0, -1, 0), (0, 0, 1)),
                ]
            ),
        ],
        # [generate_random_point(points_bank["rb_ks_monza"]) for _ in range(n)],
        [((0, 0, 0), (0, 0, 0, 0)), ((1, 1, 1), (1, 1, 1, 1))],
    ),
]

[((0, 0, 0), (0, 0, 0, 0)), ((1, 1, 1), (1, 1, 1, 1))]
0 0
0 1
1 0
1 1


In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
scenarios[0].history.save()

In [41]:
history = annotation.AnnotationScenarioHistory.load("rb_ks_monza_4")

In [45]:
print(history.locations)

[('2022__4_navaro_ssr_700_indycar', ((0, 0, 0), (0, 0, 0, 0)), [((-0.3, 1, 2), (0, -1, 0), (0, 0, 1), 70, (0.1, 1000), (224, 224))]), ('2022__4_navaro_ssr_700_indycar', ((1, 1, 1), (1, 1, 1, 1)), [((-0.3, 1, 2), (0, -1, 0), (0, 0, 1), 70, (0.1, 1000), (224, 224))]), ('boliva', ((0, 0, 0), (0, 0, 0, 0)), [((-0.3, 1, 2), (0, -1, 0), (0, 0, 1), 70, (0.1, 1000), (224, 224))]), ('boliva', ((1, 1, 1), (1, 1, 1, 1)), [((-0.3, 1, 2), (0, -1, 0), (0, 0, 1), 70, (0.1, 1000), (224, 224))])]
